In [1]:
import pandas as pd
import glob
import json
from collections import defaultdict
from flatten_dict import flatten

In [2]:
path = glob.glob('/Users/yudonghao/Desktop/The_Data_Incubator/capstone/project/data/*')

In [3]:
# testing parsing on only one file, problematic cols are the ones in the list
# problematic columns: ['Labels', 'Ratings/List', 'Vintages/List', 'Community/Reviews/List', 'ProductAttributes']
def test(p):
    df = pd.DataFrame()
    with open(p, 'r') as f:
            #get the wine list, a list of dictionaries
        wl = json.load(f)['Products']['List'] 
        for i in wl:
            #nested dictionary, flatten it
            fd = flatten(i, reducer = "path") 
            #Labels are list of a dictionary, get the dictionary out and flat the whole thing again           
            fd["Labels"] = fd['Labels'][0]       
            new = flatten(fd, reducer = "path")
            data = pd.DataFrame.from_dict(new, orient='index')
            df_list.append(data.transpose())
    return df

In [4]:
# parse all data into a data frame
# problematic columns: ['Labels', 'Ratings/List', 'Vintages/List', 'Community/Reviews/List', 'ProductAttributes']
def parse_all(path):
    df = pd.DataFrame()
    df_list = []
    for p in path:
        with open(p, 'r') as f:
            #get the wine list, a list of dictionaries
            wl = json.load(f)['Products']['List'] 
            for i in wl:
                #nested dictionary, flatten it
                fd = flatten(i, reducer = "path") 
                #Labels are list of a dictionary, get the dictionary out and flat the whole thing again           
                fd["Labels"] = fd['Labels'][0]       
                new = flatten(fd, reducer = "path")
                data = pd.DataFrame.from_dict(new, orient='index')
                df_list.append(data.transpose())
    df = pd.concat(df_list)
    return df

In [9]:
#parse all files
all_df_list = parse_all(path)

ValueError: No objects to concatenate

In [6]:
df = pd.concat(all_df_list)

In [7]:
# store the dataframe so next time don't need to run the parse script again
df.to_pickle("parsed_df.pkl")

NameError: name 'df' is not defined

In [10]:
#now load the file
v1 = pd.read_pickle("parsed_df.pkl")

In [11]:
# to see if there is anything inside the lists in the following three columns
for key in ['Ratings/List', 'Vintages/List', 'Community/Reviews/List']:
    for i in v1[key]:
        count += 1
        if len(i)!=0:
            print (v1[key], i)

NameError: name 'count' is not defined

In [ ]:
# Nothing stored there!!! Delete those three columns.
df = v1.drop(['Ratings/List', 'Vintages/List', 'Community/Reviews/List'], axis = 1)

In [ ]:
index = range(len(df))
df.index = index

In [ ]:
def parse_attr(path):
    attr = []
    for p in path:
        with open(p, 'r') as f:
            #get the wine list, a list of dictionaries
            wl = json.load(f)['Products']['List'] 
            for i in wl:
                d = flatten(i, reducer = "path")
                l = []
                for a in d['ProductAttributes']:
                    l.append(a['Name'])
                attr.append(l)
    return attr

In [ ]:
df.to_pickle("parsed_v2.pkl")

In [ ]:
v2 = pd.read_pickle("parsed_v2.pkl")

In [ ]:
#drop some useless urls
v2 = v2.drop(['Appellation/Region/Url','Community/Url','GeoLocation/Url','Varietal/WineType/Url','Vineyard/GeoLocation/Url','Vineyard/ImageUrl','Vineyard/Url','Appellation/Url','Community/Reviews/Url','Url','Varietal/Url'], axis = 1)

In [ ]:
v2.head()

In [ ]:
range(2,1)

In [ ]:
rating = v2['Ratings/HighestScore']

In [ ]:
rating = rating[rating > 0]

In [ ]:
max(rating)

In [12]:
# parse review file
def parse_review(review_path):
    with open(review_path,"r") as f:
        rev = json.load(f)
        refine = []
        for i in rev:
            if 'cust_reviews' in i.keys():
                refine.append(i)
        res = {}
        index = 0
        for i in refine:
            try:
                for j in i['cust_reviews']:
                    j['Id'] = int(i['item_number'])
                    res[index] = j
                    index += 1
            except KeyError:
                continue
    review_df = pd.DataFrame.from_dict(res, orient='index')
    return review_df

In [13]:
path = "/Users/yudonghao/Desktop/The_Data_Incubator/capstone/Python_reprocess/review/review.json"
with open(path,"r") as f:
    rev = json.load(f)
    refine = []
    for i in rev:
        if 'cust_reviews' in i.keys():
            refine.append(i)
    res = {}
    index = 0
    for i in refine:
        try:
            for j in i['cust_reviews']:
                j['Id'] = int(i['item_number'])
                res[index] = j
                index += 1
        except KeyError:
            continue

In [14]:
path = "/Users/yudonghao/Desktop/The_Data_Incubator/capstone/Python_reprocess/review/review.json"
review_df = parse_review(path)
review_df.iloc[0]['Id']

119566

In [15]:
review_df.to_pickle("review.pkl")

In [ ]:
# parse features, first look at how many features do we have
attr = v2['Attr']

In [ ]:
# get all the features in attr including wine flavors and other descriptive characteristics
att = set()
for i in attr:
    for j in i:
        att.add(j)

In [ ]:
att = list(att)

In [ ]:
att